In [1]:
import os
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image, ImageFilter

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler, Subset
from torchvision import transforms, models

from sklearn.model_selection import train_test_split

from sklearn.model_selection import KFold
from sklearn.metrics import confusion_matrix, classification_report
from tabulate import tabulate

import copy
from torch.optim.lr_scheduler import OneCycleLR

In [2]:
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed(SEED)

In [3]:
def format_confusion_matrix(cm):
    """Returns a string of a nicely formatted confusion matrix with indices and highlighted diagonal."""
    headers = [""] + [f"Pred {i}" for i in range(len(cm[0]))]
    table = []

    for i, row in enumerate(cm):
        formatted_row = []
        for j, val in enumerate(row):
            if i == j:
                formatted_row.append(f"*{val}*")  # Highlight diagonal
            else:
                formatted_row.append(str(val))
        table.append([f"True {i}"] + formatted_row)

    return tabulate(table, headers=headers, tablefmt="grid")

In [4]:
all_logs = []

def log_and_store(*msgs, table_format=False, is_confmat=False):
    """
    Logs plain messages or pretty-prints confusion matrices or tables.
    """
    if is_confmat and len(msgs) == 1 and isinstance(msgs[0], list):
        msg = format_confusion_matrix(msgs[0])
    elif table_format and len(msgs) == 1 and isinstance(msgs[0], list):
        msg = tabulate(msgs[0], tablefmt="grid")
    else:
        msg = " ".join(str(m) for m in msgs)

    print(msg)
    all_logs.append(msg)

def get_logs():
    """
    Returnerar en lista med alla loggade meddelanden.
    """
    return all_logs

def clear_logs():
    """
    Tömmer loggen.
    """
    all_logs.clear()

def save_logs_to_file(filename):
    """
    Sparar loggade meddelanden till en fil.
    """
    with open(filename, 'w') as f:
        for log in all_logs:
            f.write(log + '\n')

# Advanced Training Pipeline

This notebook implements:
1. Stronger and more varied augmentation, including class-specific oversampling.
2. Model-level adjustments: gradual unfreezing, EfficientNet-B0/B3, label smoothing, focal loss/class-weighted loss.
3. Training strategies: early stopping, checkpoint ensembles, and k-fold cross-validation.


In [5]:
# Configuration
DATA_DIR = "../datasets/data-BSS"  # Update this path
IMG_SIZE = 224
DEVICE = "cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"
NUM_CLASSES = 7


In [6]:
class StoolDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.samples = []
        self.class_to_idx = {}
        for idx, class_name in enumerate(sorted(os.listdir(root_dir))):
            class_path = os.path.join(root_dir, class_name)
            if os.path.isdir(class_path):
                self.class_to_idx[class_name] = idx
                for fname in os.listdir(class_path):
                    if fname.lower().endswith(('.png', '.jpg', '.jpeg')):
                        self.samples.append((os.path.join(class_path, fname), idx))
        
    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        img_path, label = self.samples[idx]
        image = Image.open(img_path).convert('RGB')
        if self.transform:
            image = self.transform(image)
        return image, label


In [7]:
train_transforms = transforms.Compose([
    transforms.RandomResizedCrop(IMG_SIZE, scale=(0.8, 1.0)),  # random crop + resize
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.RandomRotation(15),
    transforms.ColorJitter(brightness=0.2, contrast=0.2),
    transforms.RandomApply([transforms.GaussianBlur(3)], p=0.2),
    transforms.RandomApply([transforms.Lambda(lambda img: img.filter(ImageFilter.FIND_EDGES))], p=0.1),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

val_transforms = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [8]:
# Label Smoothing Loss (CrossEntropy with label_smoothing)
criterion_smooth = nn.CrossEntropyLoss(label_smoothing=0.1)

# Focal Loss Implementation
class FocalLoss(nn.Module):
    def __init__(self, alpha=1, gamma=2, reduction='mean'):
        super(FocalLoss, self).__init__()
        self.alpha = alpha
        self.gamma = gamma
        self.reduction = reduction

    def forward(self, inputs, targets):
        ce_loss = F.cross_entropy(inputs, targets, reduction='none')
        pt = torch.exp(-ce_loss)
        focal_loss = self.alpha * ((1 - pt) ** self.gamma) * ce_loss
        if self.reduction == 'mean':
            return focal_loss.mean()
        elif self.reduction == 'sum':
            return focal_loss.sum()
        else:
            return focal_loss
        
    def __name__(self):
        return "FocalLoss"

In [9]:
'''
def create_model(backbone='efficientnet_b0', num_classes=NUM_CLASSES, freeze_until_layer=None):
    # Load pretrained EfficientNet
    if backbone == 'efficientnet_b0':
        model = models.efficientnet_b0(pretrained=True)
    elif backbone == 'efficientnet_b3':
        model = models.efficientnet_b3(pretrained=True)
    elif backbone == 'mobilenet_v2':
        model = models.mobilenet_v2(pretrained=True)
    elif backbone == 'mobilenet_v3_small':
        model = models.mobilenet_v3_small(pretrained=True)
    else:
        raise ValueError('Invalid backbone')

    # Replace classifier head
    in_features = model.classifier[1].in_features
    model.classifier = nn.Sequential(
        nn.Linear(in_features, 512),
        nn.ReLU(inplace=True),
        nn.Dropout(0.4),
        nn.Linear(512, num_classes)
    )
    
    # Freeze layers if specified
    if freeze_until_layer:
        for name, param in model.named_parameters():
            param.requires_grad = False
            if name.startswith(freeze_until_layer):
                break
        # Unfreeze subsequent layers
        unfreeze = False
        for name, param in model.named_parameters():
            if unfreeze:
                param.requires_grad = True
            if name.startswith(freeze_until_layer):
                unfreeze = True
    
    return model.to(DEVICE)
'''


"\ndef create_model(backbone='efficientnet_b0', num_classes=NUM_CLASSES, freeze_until_layer=None):\n    # Load pretrained EfficientNet\n    if backbone == 'efficientnet_b0':\n        model = models.efficientnet_b0(pretrained=True)\n    elif backbone == 'efficientnet_b3':\n        model = models.efficientnet_b3(pretrained=True)\n    elif backbone == 'mobilenet_v2':\n        model = models.mobilenet_v2(pretrained=True)\n    elif backbone == 'mobilenet_v3_small':\n        model = models.mobilenet_v3_small(pretrained=True)\n    else:\n        raise ValueError('Invalid backbone')\n\n    # Replace classifier head\n    in_features = model.classifier[1].in_features\n    model.classifier = nn.Sequential(\n        nn.Linear(in_features, 512),\n        nn.ReLU(inplace=True),\n        nn.Dropout(0.4),\n        nn.Linear(512, num_classes)\n    )\n\n    # Freeze layers if specified\n    if freeze_until_layer:\n        for name, param in model.named_parameters():\n            param.requires_grad = Fa

In [10]:
def create_model(backbone, num_classes=NUM_CLASSES, freeze_until_layer=None):
    if backbone == 'mobilenet_v3_small':
        model = models.mobilenet_v3_small(pretrained=True)
        # freeze layers
        if freeze_until_layer:
            for name, param in model.features.named_parameters():
                param.requires_grad = False
                if freeze_until_layer in name:
                    break

        # Replace final classifier
        in_features = model.classifier[-1].in_features
        model.classifier[-1] = nn.Linear(in_features, num_classes)

    elif backbone == 'mobilenet_v3_large':
        model = models.mobilenet_v3_large(pretrained=True)
        if freeze_until_layer:
            for name, param in model.features.named_parameters():
                param.requires_grad = False
                if freeze_until_layer in name:
                    break

        in_features = model.classifier[-1].in_features
        model.classifier[-1] = nn.Linear(in_features, num_classes)

    elif backbone == 'mobilenet_v2':
        model = models.mobilenet_v2(pretrained=True)
        if freeze_until_layer:
            for name, param in model.features.named_parameters():
                param.requires_grad = False
                if freeze_until_layer in name:
                    break

        in_features = model.classifier[1].in_features
        model.classifier[1] = nn.Linear(in_features, num_classes)

    elif backbone == 'efficientnet_b0':
        model = models.efficientnet_b0(pretrained=True)
        if freeze_until_layer:
            for name, param in model.features.named_parameters():
                param.requires_grad = False
                if freeze_until_layer in name:
                    break

        in_features = model.classifier[1].in_features
        model.classifier[1] = nn.Linear(in_features, num_classes)

    elif backbone == 'efficientnet_b3':
        model = models.efficientnet_b3(pretrained=True)
        if freeze_until_layer:
            for name, param in model.features.named_parameters():
                param.requires_grad = False
                if freeze_until_layer in name:
                    break

        in_features = model.classifier[1].in_features
        model.classifier[1] = nn.Linear(in_features, num_classes)

    else:
        raise ValueError('Invalid backbone')

    return model.to(DEVICE)

In [11]:
def evaluate_model(model, loader):
    model.eval()
    all_preds = []
    all_labels = []
    with torch.no_grad():
        for inputs, labels in loader:
            inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    return None, None, all_preds, all_labels


In [12]:
def train_validate(model, train_loader, val_loader, criterion, optimizer, num_epochs, fold_idx):

    #patience = 3
    #counter = 0
    #lr_scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=3) # for accuracy
    #lr_scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=3) # for loss

    best_acc     = 0.0
    best_loss    = float('inf')
    best_weights = copy.deepcopy(model.state_dict())

    # ── One‐Cycle LR schedule ──────────────────────────────────────────────────
    scheduler = OneCycleLR(
        optimizer,
        max_lr=optimizer.param_groups[0]['lr'] * 10,  # e.g. 10× your base LR
        epochs=num_epochs,
        steps_per_epoch=len(train_loader),
    )

    for epoch in range(num_epochs):
        # Training
        model.train()
        running_loss = 0.0
        running_corrects = 0
        total = 0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item() * inputs.size(0) 
            _, preds = torch.max(outputs, 1)
            running_corrects += (preds == labels).sum().item()
            total += labels.size(0)

        epoch_loss = running_loss / total
        epoch_acc = running_corrects / total

        # Validation
        model.eval()
        val_loss = 0.0
        val_corrects = 0
        val_total = 0
        all_preds = []
        all_labels = []
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                
                val_loss += loss.item() * inputs.size(0)
                _, preds = torch.max(outputs, 1)
                val_corrects += (preds == labels).sum().item()
                val_total += labels.size(0)
                all_preds.extend(preds.cpu().numpy())
                all_labels.extend(labels.cpu().numpy())

        val_loss_epoch = val_loss / val_total
        val_acc_epoch = val_corrects / val_total
        # step the one‐cycle scheduler each batch‐cycle (done at epoch‐end here)
        scheduler.step()

        # keep snapshot of best‐ever validation loss (for final restore)
        if val_loss_epoch < best_loss:
            best_loss    = val_loss_epoch
            best_acc     = val_acc_epoch
            best_weights = copy.deepcopy(model.state_dict())

        print(f"Fold {fold_idx}, Epoch {epoch+1}/{num_epochs} - "
              f"Train Loss: {epoch_loss:.4f}, Train Acc: {epoch_acc:.4f} - "
              f"Val Loss: {val_loss_epoch:.4f}, Val Acc: {val_acc_epoch:.4f}")

        # Early stopping
        #if val_acc_epoch > best_acc:
        #    best_acc = val_acc_epoch
        #   best_weights = model.state_dict().copy()
        #   counter = 0
        #else:
        #    counter += 1
        #    if counter >= patience:
        #        print(f"Early stopping at epoch {epoch+1}")
        #       break

    # Load best weights
    model.load_state_dict(best_weights)

    # Final validation metrics
    _, _, preds, labels = evaluate_model(model, val_loader)
    log_and_store("\nClassification Report for Fold {}:".format(fold_idx))
    log_and_store(classification_report(labels, preds, target_names=sorted(os.listdir(DATA_DIR))))

    return model, best_acc


In [13]:
!find ../datasets/data-BSS -name ".DS_Store" -type f -delete

In [14]:
# Choose device
device = torch.device(
    "cuda" if torch.cuda.is_available() 
    else "mps" if getattr(torch.backends, "mps", None) and torch.backends.mps.is_available() 
    else "cpu"
)
print(f"Using device: {device}")

Using device: mps


# Training functions

In [ ]:
def run_kfold_training(
    data_dir,
    backbone='mobilenet_v2',
    freeze_until_layer=None,
    criterion_fn=None,
    num_epochs=10,
    lr=1e-4,
    k_folds=3,
    batch_size=32,
    train_transforms=None,
    val_transforms=None,
    seed=42,
    num_classes=7,
):
    full_dataset = StoolDataset(data_dir, transform=None)
    indices = list(range(len(full_dataset)))

    # Class weights for full dataset (optional, for balance insights)
    all_labels_full = [label for _, label in full_dataset]
    class_counts = np.bincount(all_labels_full)
    class_weights = 1.0 / class_counts
    weights_full = [class_weights[label] for label in all_labels_full]

    kf = KFold(n_splits=k_folds, shuffle=True, random_state=seed)
    fold_models = []
    fold_accuracies = []

    for fold_idx, (train_idx, val_idx) in enumerate(kf.split(indices), 1):
        print(f"\n======= Fold {fold_idx} =======")

        # Subset + transforms
        train_ds = torch.utils.data.Subset(StoolDataset(data_dir, transform=train_transforms), train_idx)
        val_ds   = torch.utils.data.Subset(StoolDataset(data_dir, transform=val_transforms), val_idx)

        # Weighted sampler
        train_labels_fold = [train_ds.dataset.samples[i][1] for i in train_idx]
        class_sample_count_fold = np.array([train_labels_fold.count(i) for i in range(num_classes)])
        print(f"Class sample counts: {class_sample_count_fold}")
        class_weights_fold = 1.0 / class_sample_count_fold
        sample_weights_fold = np.array([class_weights_fold[label] for label in train_labels_fold])
        sample_weights_fold = torch.from_numpy(sample_weights_fold.astype(np.double))
        sampler_fold = WeightedRandomSampler(sample_weights_fold, num_samples=len(sample_weights_fold), replacement=True)

        # DataLoaders
        train_loader = DataLoader(train_ds, batch_size=batch_size, sampler=sampler_fold)
        val_loader = DataLoader(val_ds, batch_size=batch_size, shuffle=False)

        # Create model
        model = create_model(backbone=backbone, freeze_until_layer=freeze_until_layer)
        optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=lr)

        # Loss
        criterion = criterion_fn if criterion_fn is not None else nn.CrossEntropyLoss()

        # Train
        best_model, best_acc = train_validate(model, train_loader, val_loader, criterion, optimizer, num_epochs, fold_idx)
        fold_models.append(best_model)
        fold_accuracies.append(best_acc)

        # Evaluate and log
        best_model.eval()
        all_preds, all_labels = [], []
        with torch.no_grad():
            for xb, yb in val_loader:
                xb = xb.to(device)
                logits = best_model(xb)
                preds = logits.argmax(dim=1).cpu().numpy()
                all_preds.extend(preds)
                all_labels.extend(yb.numpy())

        cm = confusion_matrix(all_labels, all_preds)
        crpt = classification_report(all_labels, all_preds, digits=4)

        log_and_store(f"\n--- Fold {fold_idx} Confusion Matrix ---")
        log_and_store(cm, is_confmat=True)

        log_and_store(f"\n--- Fold {fold_idx} Classification Report ---")
        log_and_store(crpt)

    log_and_store(["\nFold Models:", [f"Fold {i+1}" for i in range(len(fold_models))]])
    log_and_store(["Fold Accuracies:", fold_accuracies])
    log_and_store(["Mean Accuracy:", np.mean(fold_accuracies)])

    return fold_models, fold_accuracies

In [16]:
def train_single_split(
    data_dir,
    model_name='efficientnet_b3',
    freeze_until=None,
    criterion='focal',  # or 'smooth'
    batch_size=32,
    lr=1e-4,
    num_epochs=10,
    val_split=0.2,
    seed=42,
):
    print("======= Single Split Training =======")

    # Full dataset
    full_dataset = StoolDataset(data_dir, transform=None)
    indices = list(range(len(full_dataset)))
    all_labels = [label for _, label in full_dataset]

    # Train/val split
    train_idx, val_idx = train_test_split(
        indices, test_size=val_split, random_state=seed, stratify=all_labels
    )

    # Create datasets with transforms
    train_ds = Subset(StoolDataset(data_dir, transform=train_transforms), train_idx)
    val_ds = Subset(StoolDataset(data_dir, transform=val_transforms), val_idx)

    # Weighted sampler for class imbalance
    train_labels = [train_ds.dataset.samples[i][1] for i in train_idx]
    class_sample_counts = np.array([train_labels.count(i) for i in range(NUM_CLASSES)])
    print(f"Class sample counts: {class_sample_counts}")
    class_weights = 1.0 / class_sample_counts
    sample_weights = np.array([class_weights[label] for label in train_labels])
    sample_weights = torch.from_numpy(sample_weights.astype(np.double))
    sampler = WeightedRandomSampler(sample_weights, num_samples=len(sample_weights), replacement=True)

    # Data loaders
    train_loader = DataLoader(train_ds, batch_size=batch_size, sampler=sampler)
    val_loader = DataLoader(val_ds, batch_size=batch_size, shuffle=False)

    # Model
    model = create_model(backbone=model_name, freeze_until_layer=freeze_until)
    optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=lr)

    if criterion == 'focal':
        loss_fn = FocalLoss(alpha=1, gamma=2)
    elif criterion == 'smooth':
        loss_fn = criterion_smooth
    else:
        raise ValueError("Unsupported loss function")

    # Train
    best_model, best_acc = train_validate(model, train_loader, val_loader, loss_fn, optimizer, num_epochs, fold_idx=None)

    # Evaluation
    best_model.eval()
    all_preds, all_labels_eval = [], []

    with torch.no_grad():
        for xb, yb in val_loader:
            xb = xb.to(device)
            preds = best_model(xb).argmax(dim=1).cpu().numpy()
            all_preds.extend(preds)
            all_labels_eval.extend(yb.numpy())

    # Confusion matrix and classification report
    cm = confusion_matrix(all_labels_eval, all_preds)
    cr = classification_report(all_labels_eval, all_preds, digits=4)

    log_and_store("--- Confusion Matrix ---")
    log_and_store(cm, is_confmat=True)
    log_and_store("--- Classification Report ---")
    log_and_store(cr)

    return best_model, best_acc

# Training ground

In [17]:
clear_logs()  # Clear logs if needed

In [18]:
# Configuration
backbone = 'efficientnet_b3'  # or 'mobilenet_v2', 'mobilenet_v3_small', efficientnet_b3, efficientnet_b0.
freeze_until = 'features.4'  # e.g., 'features.4'
criterion = 'focal' # 'focal' or 'smooth'
num_epochs = 20
batch_size = 16 # This represents the batch size for training and validation which is the number of samples processed before the model is updated.
lr = 1e-4  # Learning rate

val_split = 0.2  # Fraction of data to use for validation

k_folds = 3
seed = 42


In [19]:
kf_models, accs = run_kfold_training(
    data_dir=DATA_DIR,
    backbone='efficientnet_b3',  # or 'mobilenet_v3_small', 'efficientnet_b0', 'efficientnet_b3'
    freeze_until_layer=None,  # or 'features.4' for EfficientNet
    criterion_fn=FocalLoss(alpha=1, gamma=2),
    num_epochs=25,
    lr=1e-4,
    k_folds=5,
    batch_size=16,
    train_transforms=train_transforms,
    val_transforms=val_transforms,
    seed=4,
    num_classes=7,
)


======= Fold 1 =======


/Users/sebastianapelgren/code/poop-ai/.venv/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/sebastianapelgren/code/poop-ai/.venv/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=EfficientNet_B3_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_B3_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Fold 1, Epoch 1/25 - Train Loss: 1.3894, Train Acc: 0.1985 - Val Loss: 1.3430, Val Acc: 0.2938
Fold 1, Epoch 2/25 - Train Loss: 1.3055, Train Acc: 0.3041 - Val Loss: 1.2449, Val Acc: 0.3918
Fold 1, Epoch 3/25 - Train Loss: 1.1855, Train Acc: 0.3814 - Val Loss: 1.1397, Val Acc: 0.4639
Fold 1, Epoch 4/25 - Train Loss: 1.0984, Train Acc: 0.4446 - Val Loss: 1.0660, Val Acc: 0.4536
Fold 1, Epoch 5/25 - Train Loss: 1.0050, Train Acc: 0.5013 - Val Loss: 1.0056, Val Acc: 0.4691
Fold 1, Epoch 6/25 - Train Loss: 0.9427, Train Acc: 0.5155 - Val Loss: 0.9152, Val Acc: 0.4691
Fold 1, Epoch 7/25 - Train Loss: 0.8515, Train Acc: 0.5451 - Val Loss: 0.8592, Val Acc: 0.4330
Fold 1, Epoch 8/25 - Train Loss: 0.7587, Train Acc: 0.5851 - Val Loss: 0.7459, Val Acc: 0.5309
Fold 1, Epoch 9/25 - Train Loss: 0.6569, Train Acc: 0.6469 - Val Loss: 0.7472, Val Acc: 0.5361
Fold 1, Epoch 10/25 - Train Loss: 0.5770, Train Acc: 0.6856 - Val Loss: 0.7018, Val Acc: 0.5206
Fold 1, Epoch 11/25 - Train Loss: 0.5606, Train A

/Users/sebastianapelgren/code/poop-ai/.venv/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/sebastianapelgren/code/poop-ai/.venv/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=EfficientNet_B3_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_B3_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Fold 2, Epoch 1/25 - Train Loss: 1.3934, Train Acc: 0.1946 - Val Loss: 1.3612, Val Acc: 0.2113
Fold 2, Epoch 2/25 - Train Loss: 1.2941, Train Acc: 0.3222 - Val Loss: 1.3038, Val Acc: 0.3144
Fold 2, Epoch 3/25 - Train Loss: 1.1913, Train Acc: 0.3737 - Val Loss: 1.2145, Val Acc: 0.3402
Fold 2, Epoch 4/25 - Train Loss: 1.0722, Train Acc: 0.4704 - Val Loss: 1.1349, Val Acc: 0.3763
Fold 2, Epoch 5/25 - Train Loss: 0.9851, Train Acc: 0.4897 - Val Loss: 1.0708, Val Acc: 0.4124
Fold 2, Epoch 6/25 - Train Loss: 0.8655, Train Acc: 0.5464 - Val Loss: 0.9729, Val Acc: 0.4227
Fold 2, Epoch 7/25 - Train Loss: 0.7838, Train Acc: 0.5799 - Val Loss: 0.8844, Val Acc: 0.4433
Fold 2, Epoch 8/25 - Train Loss: 0.7271, Train Acc: 0.5941 - Val Loss: 0.8369, Val Acc: 0.5000
Fold 2, Epoch 9/25 - Train Loss: 0.5993, Train Acc: 0.6856 - Val Loss: 0.8093, Val Acc: 0.4845
Fold 2, Epoch 10/25 - Train Loss: 0.5646, Train Acc: 0.6572 - Val Loss: 0.7392, Val Acc: 0.5155
Fold 2, Epoch 11/25 - Train Loss: 0.5343, Train A

...

Fold 3, Epoch 1/25 - Train Loss: 1.4076, Train Acc: 0.1869 - Val Loss: 1.3993, Val Acc: 0.2113
Fold 3, Epoch 2/25 - Train Loss: 1.3135, Train Acc: 0.2887 - Val Loss: 1.3324, Val Acc: 0.2577
Fold 3, Epoch 3/25 - Train Loss: 1.2205, Train Acc: 0.3570 - Val Loss: 1.2609, Val Acc: 0.2990
Fold 3, Epoch 4/25 - Train Loss: 1.1191, Train Acc: 0.4472 - Val Loss: 1.1531, Val Acc: 0.3711
Fold 3, Epoch 5/25 - Train Loss: 1.0145, Train Acc: 0.4678 - Val Loss: 1.0613, Val Acc: 0.4588
Fold 3, Epoch 6/25 - Train Loss: 0.9063, Train Acc: 0.5438 - Val Loss: 0.9748, Val Acc: 0.4845
Fold 3, Epoch 7/25 - Train Loss: 0.8154, Train Acc: 0.5593 - Val Loss: 0.8847, Val Acc: 0.5155
Fold 3, Epoch 8/25 - Train Loss: 0.7218, Train Acc: 0.5992 - Val Loss: 0.8389, Val Acc: 0.5052
Fold 3, Epoch 9/25 - Train Loss: 0.6774, Train Acc: 0.6211 - Val Loss: 0.7805, Val Acc: 0.5619
Fold 3, Epoch 10/25 - Train Loss: 0.5490, Train Acc: 0.6946 - Val Loss: 0.7393, Val Acc: 0.5206
Fold 3, Epoch 11/25 - Train Loss: 0.5316, Train A

/Users/sebastianapelgren/code/poop-ai/.venv/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/sebastianapelgren/code/poop-ai/.venv/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=EfficientNet_B3_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_B3_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Fold 4, Epoch 1/25 - Train Loss: 1.4073, Train Acc: 0.1830 - Val Loss: 1.3746, Val Acc: 0.2371
Fold 4, Epoch 2/25 - Train Loss: 1.3196, Train Acc: 0.3080 - Val Loss: 1.2966, Val Acc: 0.3041
Fold 4, Epoch 3/25 - Train Loss: 1.2162, Train Acc: 0.3943 - Val Loss: 1.2038, Val Acc: 0.3969
Fold 4, Epoch 4/25 - Train Loss: 1.1193, Train Acc: 0.4291 - Val Loss: 1.1063, Val Acc: 0.4639
Fold 4, Epoch 5/25 - Train Loss: 1.0135, Train Acc: 0.5103 - Val Loss: 0.9974, Val Acc: 0.5206
Fold 4, Epoch 6/25 - Train Loss: 0.9175, Train Acc: 0.5322 - Val Loss: 0.9172, Val Acc: 0.5103
Fold 4, Epoch 7/25 - Train Loss: 0.7946, Train Acc: 0.5644 - Val Loss: 0.8448, Val Acc: 0.5103
Fold 4, Epoch 8/25 - Train Loss: 0.7251, Train Acc: 0.6044 - Val Loss: 0.7895, Val Acc: 0.5000
Fold 4, Epoch 9/25 - Train Loss: 0.6530, Train Acc: 0.6198 - Val Loss: 0.7279, Val Acc: 0.5309
Fold 4, Epoch 10/25 - Train Loss: 0.6013, Train Acc: 0.6637 - Val Loss: 0.7492, Val Acc: 0.5155
Fold 4, Epoch 11/25 - Train Loss: 0.4801, Train A

/Users/sebastianapelgren/code/poop-ai/.venv/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/sebastianapelgren/code/poop-ai/.venv/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=EfficientNet_B3_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_B3_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Fold 5, Epoch 1/25 - Train Loss: 1.4184, Train Acc: 0.1585 - Val Loss: 1.3740, Val Acc: 0.2887
Fold 5, Epoch 2/25 - Train Loss: 1.2942, Train Acc: 0.3273 - Val Loss: 1.2893, Val Acc: 0.3866
Fold 5, Epoch 3/25 - Train Loss: 1.2058, Train Acc: 0.4046 - Val Loss: 1.2039, Val Acc: 0.4639
Fold 5, Epoch 4/25 - Train Loss: 1.1240, Train Acc: 0.4227 - Val Loss: 1.1243, Val Acc: 0.4536
Fold 5, Epoch 5/25 - Train Loss: 1.0163, Train Acc: 0.4884 - Val Loss: 1.0244, Val Acc: 0.5309
Fold 5, Epoch 6/25 - Train Loss: 0.9089, Train Acc: 0.5284 - Val Loss: 0.9524, Val Acc: 0.5103
Fold 5, Epoch 7/25 - Train Loss: 0.8012, Train Acc: 0.6005 - Val Loss: 0.8396, Val Acc: 0.5515
Fold 5, Epoch 8/25 - Train Loss: 0.7174, Train Acc: 0.6095 - Val Loss: 0.7604, Val Acc: 0.5619
Fold 5, Epoch 9/25 - Train Loss: 0.6712, Train Acc: 0.6044 - Val Loss: 0.7484, Val Acc: 0.5876
Fold 5, Epoch 10/25 - Train Loss: 0.5882, Train Acc: 0.6546 - Val Loss: 0.7024, Val Acc: 0.5876
Fold 5, Epoch 11/25 - Train Loss: 0.5402, Train A

In [109]:
best_model, best_acc = train_single_split(
    data_dir=DATA_DIR,
    model_name='mobilenet_v3_large',  # or 'mobilenet_v3_small', 'efficientnet_b0', 'efficientnet_b3'
    freeze_until=None,  # or 'features.4', None for no freezing
    criterion='focal',  # or 'smooth'
    batch_size=16,
    lr=1e-4,
    num_epochs=15,
    val_split=0.2
)

======= Single Split Training =======
Class sample counts: [ 85 110 270 263  31  34  67]


/Users/sebastianapelgren/code/poop-ai/.venv/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/sebastianapelgren/code/poop-ai/.venv/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V3_Large_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V3_Large_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Fold None, Epoch 1/15 - Train Loss: 1.3483, Train Acc: 0.2779 - Val Loss: 1.2287, Val Acc: 0.3116
Fold None, Epoch 2/15 - Train Loss: 1.1277, Train Acc: 0.3791 - Val Loss: 0.9899, Val Acc: 0.4093
Fold None, Epoch 3/15 - Train Loss: 0.9610, Train Acc: 0.4767 - Val Loss: 0.8523, Val Acc: 0.4512
Fold None, Epoch 4/15 - Train Loss: 0.7984, Train Acc: 0.5744 - Val Loss: 0.7394, Val Acc: 0.5302
Fold None, Epoch 5/15 - Train Loss: 0.6748, Train Acc: 0.6081 - Val Loss: 0.6649, Val Acc: 0.5488
Fold None, Epoch 6/15 - Train Loss: 0.5889, Train Acc: 0.6500 - Val Loss: 0.6483, Val Acc: 0.5628
Fold None, Epoch 7/15 - Train Loss: 0.5455, Train Acc: 0.6721 - Val Loss: 0.6470, Val Acc: 0.5395
Fold None, Epoch 8/15 - Train Loss: 0.4737, Train Acc: 0.7012 - Val Loss: 0.6112, Val Acc: 0.5814
Fold None, Epoch 9/15 - Train Loss: 0.4449, Train Acc: 0.7174 - Val Loss: 0.5528, Val Acc: 0.6279
Fold None, Epoch 10/15 - Train Loss: 0.3791, Train Acc: 0.7337 - Val Loss: 0.5617, Val Acc: 0.6140
Fold None, Epoch 11

In [ ]:
best_model, best_acc = train_single_split(
    data_dir=DATA_DIR,
    model_name='efficientnet_b3',
    freeze_until=None,  # or 'features.4', None for no freezing. A higher layer means more layers are frozen.
    criterion='focal', # 'focal' or 'smooth'
    batch_size=32,
    lr=1e-4,
    num_epochs=20,
    val_split=0.2
)

======= Single Split Training =======
Class sample counts: [100 111 270 262  31  34  67]


/Users/sebastianapelgren/code/poop-ai/.venv/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/sebastianapelgren/code/poop-ai/.venv/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=EfficientNet_B3_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_B3_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Fold None, Epoch 1/15 - Train Loss: 1.4252, Train Acc: 0.1497 - Val Loss: 1.3597, Val Acc: 0.2557
Fold None, Epoch 2/15 - Train Loss: 1.3566, Train Acc: 0.2480 - Val Loss: 1.3260, Val Acc: 0.3059
Fold None, Epoch 3/15 - Train Loss: 1.3074, Train Acc: 0.2926 - Val Loss: 1.2779, Val Acc: 0.3562
Fold None, Epoch 4/15 - Train Loss: 1.2326, Train Acc: 0.3554 - Val Loss: 1.2260, Val Acc: 0.4018
Fold None, Epoch 5/15 - Train Loss: 1.1748, Train Acc: 0.3851 - Val Loss: 1.1556, Val Acc: 0.4064
Fold None, Epoch 6/15 - Train Loss: 1.1128, Train Acc: 0.4320 - Val Loss: 1.0714, Val Acc: 0.4521
Fold None, Epoch 7/15 - Train Loss: 0.9950, Train Acc: 0.4983 - Val Loss: 0.9776, Val Acc: 0.4886
Fold None, Epoch 8/15 - Train Loss: 0.9359, Train Acc: 0.5280 - Val Loss: 0.9076, Val Acc: 0.4977
Fold None, Epoch 9/15 - Train Loss: 0.8530, Train Acc: 0.5543 - Val Loss: 0.8295, Val Acc: 0.5342
Fold None, Epoch 10/15 - Train Loss: 0.7801, Train Acc: 0.5749 - Val Loss: 0.7839, Val Acc: 0.5297
Fold None, Epoch 11

In [ ]:
# Prestera inte jätte bra. mobilenet_v2 är bättre 

best_model, best_acc = train_single_split(
    data_dir=DATA_DIR,
    model_name='mobilenet_v3_large',
    freeze_until='features.7',  # or 'features.4', None for no freezing.
    criterion='focal', # 'focal' or 'smooth'
    batch_size=32,
    lr=1e-4,
    num_epochs=20,
    val_split=0.2
)

======= Single Split Training =======
Class sample counts: [100 111 270 262  31  34  67]


/Users/sebastianapelgren/code/poop-ai/.venv/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/sebastianapelgren/code/poop-ai/.venv/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V3_Large_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V3_Large_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Fold None, Epoch 1/20 - Train Loss: 1.4120, Train Acc: 0.1749 - Val Loss: 1.2051, Val Acc: 0.3425
Fold None, Epoch 2/20 - Train Loss: 1.2750, Train Acc: 0.2720 - Val Loss: 1.1779, Val Acc: 0.3470
Fold None, Epoch 3/20 - Train Loss: 1.1859, Train Acc: 0.3154 - Val Loss: 1.1232, Val Acc: 0.3790
Fold None, Epoch 4/20 - Train Loss: 1.1256, Train Acc: 0.3749 - Val Loss: 1.0569, Val Acc: 0.3836
Fold None, Epoch 5/20 - Train Loss: 1.0594, Train Acc: 0.4206 - Val Loss: 1.0140, Val Acc: 0.3927
Fold None, Epoch 6/20 - Train Loss: 0.9753, Train Acc: 0.4720 - Val Loss: 0.9710, Val Acc: 0.4018
Fold None, Epoch 7/20 - Train Loss: 0.9627, Train Acc: 0.4583 - Val Loss: 0.9287, Val Acc: 0.4521
Fold None, Epoch 8/20 - Train Loss: 0.9476, Train Acc: 0.4514 - Val Loss: 0.8917, Val Acc: 0.4840
Fold None, Epoch 9/20 - Train Loss: 0.8636, Train Acc: 0.5177 - Val Loss: 0.8702, Val Acc: 0.4703
Fold None, Epoch 10/20 - Train Loss: 0.8627, Train Acc: 0.4949 - Val Loss: 0.8426, Val Acc: 0.4932
Fold None, Epoch 11

In [150]:
logs = get_logs()

print("==== ALL LOGS ====")
for log in logs:
    print(log)

==== ALL LOGS ====

Classification Report for Fold None:
              precision    recall  f1-score   support

      type-1       0.72      0.72      0.72        25
      type-2       0.55      0.72      0.62        25
      type-3       0.53      0.49      0.51        65
      type-4       0.72      0.64      0.68        61
      type-5       0.50      0.83      0.62         6
      type-6       0.60      0.60      0.60         5
      type-7       0.80      0.80      0.80        15

    accuracy                           0.63       202
   macro avg       0.63      0.69      0.65       202
weighted avg       0.64      0.63      0.63       202



# To load existing model

In [78]:
def load_model(model_path):
    model = models.efficientnet_b0()
    in_features = model.classifier[1].in_features
    model.classifier = nn.Sequential(
        nn.Linear(in_features, 512),
        nn.ReLU(inplace=True),
        nn.Dropout(0.4),
        nn.Linear(512, NUM_CLASSES)
    )
    state_dict = torch.load(model_path, map_location=DEVICE)
    model.load_state_dict(state_dict)
    model.to(DEVICE)
    model.eval()
    return model

In [80]:
MODEL_PATH = "../api/stool_model.pth"


# model = load_model(MODEL_PATH)

# Save model

### Save as .pth

In [ ]:
# 1. Define where to save
SAVE_PATH = "../api/stool_model.pth"

# 2. Save the state_dict
#torch.save(model.state_dict(), SAVE_PATH)
print(f"Model weights saved to {SAVE_PATH}")

Model weights saved to stool_model.pth


### Save as .onnx

In [76]:
model.eval()

# Create dummy input for ONNX export (batch_size=1, 3 channels, IMG_SIZE x IMG_SIZE)
dummy_input = torch.randn(1, 3, IMG_SIZE, IMG_SIZE).to(device)

# Export the model
torch.onnx.export(
    model,                               # your trained model
    dummy_input,                         # input tensor
    "stool_model.onnx",                  # output file name
    export_params=True,                  # store weights inside the model file
    opset_version=11,                    # ONNX opset version
    do_constant_folding=True,            # fold constant values for optimization
    input_names=['input'],               # name for the input layer
    output_names=['output'],             # name for the output layer
    dynamic_axes={                      # allow variable input sizes
        'input': {0: 'batch_size'},     
        'output': {0: 'batch_size'}
    }
)

print("✅ ONNX export completed: stool_model.onnx")

✅ ONNX export completed: stool_model.onnx
